# Add/update embedding vectors


When adding a new set of embedding vectors or updating them, we need to perform the following sequence of steps

1. Given a model id, its revision, and a given a set of resources, ask the service[or some python code] for embedding vectors
2. Create/update embedding resources according to [this mapping](https://bbpgitlab.epfl.ch/dke/users/eugeniashurko/dataset-embeddings/-/blob/master/mappings/seu-embedding.hjson) --> model revision needs to be added to the `generation.activity.used.id`
3. Push them to Nexus
4. Tag them with the model UUID and the its revision (e.g. `e2b953b9-6724-4278-a1e5-3472bd63e374?rev=1`)

Related JIRA tickets: 
* https://bbpteam.epfl.ch/project/issues/browse/DKE-718
* https://bbpteam.epfl.ch/project/issues/browse/DKE-715

Prerequisites:

- The embedding model has been built
- Embedding service can read models from a dedicated Nexus project where all models are stored (here, at the moment, we can download models locally and get vectors directly from the models, without using the service)
- Model ID equals the Nexus resource id of the EmbeddingModel resource
- __Important__: local contexts in the projects with vectors should contain:

```
{
      "embedding": {
        "@id": "nsg:embedding",
        "@container": "@list"
      }
}
```

Questions:

* do we really need to url-encode tags ?
* add missing types and properties to the context

---

## Setup

### Imports

In [1]:
import requests
import getpass
import uuid
import os
import math
import warnings

from collections import OrderedDict

import numpy as np
import nexussdk as nxs

from collections import namedtuple
from urllib.parse import quote_plus
from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.mappings import DictionaryMapping
from bluegraph.downstream import EmbeddingPipeline
from bluegraph.core import GraphElementEmbedder

In [2]:
from kgforge.version import __version__
print(__version__)

0.7.1.dev1+g168ed9c


### Helpers

In [81]:
BucketConfiguration = namedtuple(
    'BucketConfiguration', 'endpoint org proj')


def create_forge_session(bucket_config):
    return KnowledgeGraphForge(
        "../../configs/new-forge-config.yaml",
        token=TOKEN, 
        endpoint=bucket_config.endpoint,        
        bucket=f"{bucket_config.org}/{bucket_config.proj}")


def register_embeddings(forge, vectors, model_id, model_revision, tag):
    new_embeddings = []
    updated_embeddings = []
    for at_id, embedding in vectors.items():
        existing_vectors = forge.search({
            "type": "Embedding",
            "derivation": {
                "entity": {
                    "id": at_id
                }
            },
            "generation": {
                "activity": {
                    "used": {
                        "id": model_id
                    }
                }
            }
        })
        if existing_vectors:
            vector_resource = existing_vectors[0]
            vector_resource.embedding = embedding
            vector_resource.generation.activity.used.hasSelector = forge.from_json({
                "type": "FragmentSelector",
                "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                "value": f"?rev={model_revision}"
            })
            updated_embeddings.append(vector_resource)
        else: 
            new_embeddings.append({
                "morphology_id": at_id,
                "morphology_rev": "TODO",
                "model_id": model_id,
                "model_rev": model_revision,
                "embedding_name": f"Embedding of morphology {at_id.split('/')[-1]} at revision TODO" ,
                "embedding": embedding,
                "uuid": at_id.split("/")[-1]

            })
    mapping = DictionaryMapping.load("../../mappings/seu-embedding.hjson")
    new_embedding_resources = forge.map(new_embeddings, mapping)
    for r in new_embedding_resources:
        r.id = forge.format("identifier", "embeddings", str(uuid.uuid4()))
#    # There is some error with registering new resources, so instead of
#     forge.register(new_embedding_resources)
#     print("Tagging new resources...")
#     forge.tag(new_embedding_resources, tag)
#   # I do the following:
    new_embedding_resources = forge.from_json(forge.as_json(new_embedding_resources))
    forge.register(new_embedding_resources)
    for r in new_embedding_resources:
        forge.tag(forge.retrieve(r.id), tag)
    
    forge.update(updated_embeddings)
    print("Tagging updated resources...")
    forge.tag(updated_embeddings, tag)

---

## User input

In [82]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"
DOWNLOAD_DIR = "../../data"
TOKEN = getpass.getpass()

········


Bucket where embedding models live

In [83]:
MODEL_CATALOG_ORG = "dke"
MODEL_CATALOG_PROJECT = "embedding-pipelines"

ID of the embedding model to use.

In [84]:
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/d0c21fd5-cb9c-445c-b0a4-94847ba61f5a  # neurite features
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/9fe6873b-ef6a-41b5-854a-382bc1be9fff  # dendrite
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/84519407-ad30-4d31-877e-1d6560325393  # axon
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/1c4fcd2e-000f-437b-b65b-844ee211105a  # brain regions
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/608fab85-0cc9-4ff9-a4bd-4249589b5889  # coordinates
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/43965be4-72f9-4901-9a95-d9ca13da8fb4  # TMD
# https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/7a111efa-7467-42d2-9e0c-c1ca7a883216  # TMD (scaled)

__PROVIDE HERE THE ID OF YOUR MODEL (OPTIONAL, REVISION)__

In [86]:
MODEL_ID = "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/7a111efa-7467-42d2-9e0c-c1ca7a883216"
MODEL_REVISION = None  # Specify a revision, if necessary. If None, the latest revision is used

Buckets where the input data lives together with the Bucket where the new embedding vectors should be registered.

In [87]:
DATA_BUCKETS = {
    BucketConfiguration("https://bbp.epfl.ch/nexus/v1", "bbp-external", "seu"): [
             BucketConfiguration(
                "https://bbp.epfl.ch/nexus/v1",
                 "dke","seu-embeddings")
        ]
}

If the embedding endpoint/bucket are not specified, we assume that embeddings should live in the same bucket as the input data.

Data type filter for generating embedding vectors

In [88]:
DATA_TYPE_FILTER = "NeuronMorphology"

In [89]:
HARD_RESOURCE_LIMIT = 10000  # Limit on number of resources we can retrieve with SPARQL queries

---

## Create Forge sessions

### Session for embedding models

In [90]:
forge_models = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{MODEL_CATALOG_ORG}/{MODEL_CATALOG_PROJECT}")

### Sessions for different buckets for data and embedding vectors

In [91]:
# TODO: find a way to pass different tokens and different configs
FORGE_SESSIONS = {}
for data_bucket, emb_buckets in DATA_BUCKETS.items():
    if data_bucket not in FORGE_SESSIONS:
        FORGE_SESSIONS[data_bucket] = create_forge_session(data_bucket)
    for bucket in emb_buckets:
        if bucket not in FORGE_SESSIONS:
            FORGE_SESSIONS[bucket] = create_forge_session(bucket)

---

## Fetch resources from data buckets

In [42]:
resource_set = {}
for bucket_config in DATA_BUCKETS.keys():
    if bucket_config not in resource_set:
#         !!! CURRENTLY SEARCH DOES NOT WORK, WHEN TOO MANY RESOURCES
#
#         resources = DATA_SESSIONS[
#             (bucket.data_endpoint, bucket.data_org, bucket.data_proj)].search(
#                 {"type": DATA_TYPE_FILTER}, limit=None)
        forge = FORGE_SESSIONS[bucket_config]
        query = f"""
            SELECT ?id
            WHERE {{
                ?id a {DATA_TYPE_FILTER} ;
                    <https://bluebrain.github.io/nexus/vocabulary/deprecated> false .
            }}
        """ 
        resources = forge.sparql(query, limit=HARD_RESOURCE_LIMIT)
        resources = [forge.retrieve(r.id) for r in resources] 

        resource_set[bucket_config] = resources

In [43]:
for k, v in resource_set.items():
    print("Bucket: ", k)
    print("\t", len(v), "resources")
    print()

Bucket:  BucketConfiguration(endpoint='https://bbp.epfl.ch/nexus/v1', org='bbp-external', proj='seu')
	 298 resources



## Load the embedding model

In [92]:
model_resource = forge_models.retrieve(
    f"{MODEL_ID}{'?rev=' + str(MODEL_REVISION) if MODEL_REVISION is not None else ''}")

# If revision is not provided by the user, fetch the latest
if MODEL_REVISION is None:
    MODEL_REVISION = model_resource._store_metadata._rev 

MODEL_TAG = f"{MODEL_ID.split('/')[-1]}?rev={MODEL_REVISION}"

In [93]:
MODEL_TAG

'7a111efa-7467-42d2-9e0c-c1ca7a883216?rev=4'

In [94]:
forge_models.download(model_resource, "distribution.contentUrl", DOWNLOAD_DIR, overwrite=True)
pipeline_path = os.path.join(DOWNLOAD_DIR, model_resource.distribution.name)

In [95]:
pipeline = EmbeddingPipeline.load(
    pipeline_path,
    embedder_interface=GraphElementEmbedder,
    embedder_ext="zip")

In [96]:
embedding_table = pipeline.generate_embedding_table()

## Compute embedding vectors for all the resources and push to Nexus

- TODO: add the NeuronMorphology revision once available
- TODO: add prediction of previously unseen points (currently, only the in-sample points are considered)

In [99]:
for bucket_config, resources in resource_set.items():
    vectors = {}
    for resource in resources:
        if resource.id not in vectors:
            if resource.id in embedding_table.index:
                vectors[resource.id] = embedding_table.loc[
                    resource.id].tolist()[0].tolist()
            else:
                warnings.warn(
                    f"Embedding vector for '{resource.id}' in '{bucket_config}' was not computed")
    for embedding_bucket in DATA_BUCKETS[bucket_config]:
        forge = FORGE_SESSIONS[embedding_bucket]
        print(f"Registering/updating {len(vectors)} vectors for '{embedding_bucket}...'")
        register_embeddings(forge, vectors, MODEL_ID, MODEL_REVISION, MODEL_TAG)

/Users/oshurko/opt/anaconda3/envs/bg/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Embedding vector for 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/88deb1fe-8edf-4ffc-8973-8240dea2a3d6' in 'BucketConfiguration(endpoint='https://bbp.epfl.ch/nexus/v1', org='bbp-external', proj='seu')' was not computed
  # Remove the CWD from sys.path while we load stuff.
/Users/oshurko/opt/anaconda3/envs/bg/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Embedding vector for 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d96779ca-fa70-490b-af1a-de1f8cc21d3b' in 'BucketConfiguration(endpoint='https://bbp.epfl.ch/nexus/v1', org='bbp-external', proj='seu')' was not computed
  # Remove the CWD from sys.path while we load stuff.
/Users/oshurko/opt/anaconda3/envs/bg/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Embedding vector for 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/52da11e9-6f18-42dd-88f

Registering/updating 295 vectors for 'BucketConfiguration(endpoint='https://bbp.epfl.ch/nexus/v1', org='dke', proj='seu-embeddings')...'

<count> 295
<action> _update_many
<succeeded> True
Tagging new resources...

Tagging updated resources...
<count> 295
<action> _tag_many
<succeeded> True


The following tag should be used to create new ES views on the vectors.

In [101]:
MODEL_TAG

'7a111efa-7467-42d2-9e0c-c1ca7a883216?rev=4'